# Part 2: A Simple Agent Based Model In Python


## Summary
In this second tutorial, we will build our first simple network of agents using the commands in the first notebook.
We will create a network of multiple simple agents who interact in random dyads, and simulate changes in the community over multiple interactions. 
In each interaction, one agent (the producer) produces one of two possible vowel variants according to their existing representations. Then, the second agent (the listener) can either stick to their existing variant, or adapt to the producer by changing their vowel accordingly. The listener's behavior is based on biased "personalities": agents can be flexible (adapt to their partner) or stubborn (not adapt).
We repeat this process multiple times and see what happens to the variants in the population at large. We will try to answer questions like: Has one of the variants spread to the entire community? Does this depend on the community's size and inital structure? How many stubborn people must be present to prevent (or faciliate?) convergence? etc.

-------------- 


### 1. Setting the network
First, let's create lists containing the possible parameters for our agents. We will create a seperate list for the possible vowel representations and possible personalties our agents can have:

In [1]:
# Setting the parameters

vowels = ['a', 'i']

personalities = ['F', 'S'] # F= Flexible, S=Stubborn

And let's write a simple function to create agents. An agent is just represented as a list with two values (a vowel and a personality).

In [2]:
def make_agent(vowel, personality):
    return [vowel, personality]

# For examaple, we can create a flexible agent with the vowel 'i' using our function

agent_one = make_agent(vowels[1], personalities[0])
print(agent_one)

['i', 'F']


Now, we can write functions that make populations of N agents (again, in the form of a list):

In [5]:
# Create a function that generates a population of N identical agents with given parameters

def make_population_identical(N):
    
    population = []
    
    for i in range(N):
        
        agent = make_agent(vowels[1], personalities[0])
        
        population.append(agent)

    return population


# Call the function to make a population of 5 identical agents

population_test = make_population_identical(5)
print(population_test)

[['i', 'F'], ['i', 'F'], ['i', 'F'], ['i', 'F'], ['i', 'F']]


In [6]:
# Create a function that generates of population of N agents with randomly selected parameters from each list
# using "random.choice()"

import random

def make_population_random(N):
    
    population = []
    
    for i in range(N):
        
        v = random.choice(vowels)
        
        p = random.choice(personalities)
        
        agent = make_agent(v, p)
        
        population.append(agent)

    return population

You can run the box of code below multiple times to make sure you are really getting random populations:

In [9]:
# Call the function to make a population of 5 random agents
population = make_population_random(5)
print(population)

[['a', 'F'], ['i', 'F'], ['a', 'F'], ['a', 'S'], ['i', 'F']]


In [11]:
# You can achieve the same goal using "random.int()" and using the index of the lists of possible parameters

def make_population(N):
    
    population = []
    
    for i in range(N):
        
        v = random.randint(0,1)
        
        p = random.randint(0,1)
        
        agent = make_agent(vowels[v], personalities[p])
        
        population.append(agent)

    return population

# Call the funtion and make a population of 8 random agents
# You can play with the numbers to make bigger or smaller populations
pop = make_population(8)
print(pop)

[['a', 'F'], ['a', 'F'], ['a', 'F'], ['a', 'F'], ['a', 'S'], ['i', 'S'], ['a', 'F'], ['a', 'S']]


In [12]:
# Create a function that calculates the proportion of agents with the variant 'a' in the population

def count(population):
    t = 0. # must be a float!     
    for agent in population:
        if agent[0] == 'a':
            t += 1            # The syntax =+ Adds 1 to t (or: t = t + 1)
    return t / len(population)

For a given population, we can now check how many agents are using each possible vowel variant. This is important, because later we'll also want to see how the proportion of each variant changes over the course of multiple interactions.

For this, we'll write a function that calculates the proportion of a given vowel in a population:

In [18]:
# Call the funtion on a population of 20 random agents
# You can run this box multiple times to see the proportion in different populations of different sizes

prop_a = count(make_population(20))

print('The proportion of [a] in the population is', prop_a)

The proportion of [a] in the population is 0.6


### 2. Interaction time!
We have a population, and now we want the agents to interact with each other. 

So first, we need to make a function that randomly selects two agents from the population:

In [25]:
from numpy.random import choice

def choose_pair(population):
    i = random.randint(0, len(population) - 1) # phyton counts from 0, so pop(8) is an error
    j = random.randint(0, len(population) - 1)
    
    while i == j:
        j = random.randint(0, len(population) - 1) # make sure the same agent is not selected twice
        
    return population[i], population[j]


# And we'll test it to see that really does what we want
# You can run this box of code multiple times to make sure you are really getting random pairs

pop = make_population(8)
listener, producer = choose_pair(pop)

print('The population is', population)
print('This is the chosen pair', listener, producer)
print('The listener is', listener)
print('The producer is', producer)

The population is [['a', 'F'], ['i', 'F'], ['a', 'F'], ['a', 'S'], ['i', 'F']]
This is the chosen pair ['i', 'F'] ['a', 'F']
The listener is ['i', 'F']
The producer is ['a', 'F']


Now, let's write a function that makes this pair "interact"!

If the producer and listener have the same vowel representation, nothing changes. If the agents have different vowels, then the listener's action depends on their prior personality: if they are stubborn, they will not change their vowel; but if they are flexible, they will update their vowel according to the producer.

So if the listener is flexible and has a different variant than the producer, we want to update the listener's vowel based on the producer's vowel.

To do this, we'll need to use a function called "deepcopy" to make a copy of the producer rather than pointing to the producer itself, because otherwise Python will have these two agents linked togeher forever. This is of course unwanted, since we want to update the listener only once based on a single interaction. Therefore, we'll use function called "deepcopy", which basically does what we want except for not linking the actual agents together.

In [26]:
from copy import deepcopy

def interact_test(listener,producer): 
    
    if listener[0] == producer[0]:
        return listener # if the listener and producer have the same vowel, no change
    else:
        if listener[1]=='S':
            return listener # if the listener is stubborn, no change
        else:
            listener[0]=deepcopy(producer[0])
            return listener

You can check the output of the loop by running the code line below multiple times:

In [28]:
randomlistener, randomproducer = choose_pair(make_population(8))

print('The listener is', randomlistener)
print('The producer is', randomproducer)

updated_listener = interact_test(randomlistener, randomproducer)

print('After ineracting, the listener is',updated_listener)

The listener is ['i', 'F']
The producer is ['i', 'S']
After ineracting, the listener is ['i', 'F']


So now we have a tested function that updates agents after interaction. Since we don't actually need the function to return the listener as output, we can change it to have no output, only to update the agents if needed:

In [29]:
# Create a function that only updates agents using "pass" (which means do nothing in Python)

def interact(listener,producer): 
    
    if listener[0] == producer[0]:
        pass   # do nothing
    else:
        if listener[1]=='S':
            pass
        else:
            listener[0]=deepcopy(producer[0])

Ok, we're almost ready to run some simulations! So far, we've created a a few basic functions: 
1. Make Agent - creating an agent that has a vowel and a personality
2. Make Population - creating a population of agents using the function in (1)
3. Count - counting the proportion of agents with the same vowel in the population created by (2)
4. Choose Pair - choosing two agents out of the population created in (2)
5. Interact - implementing the interaxction between the two agents chosen in (4)

Great job! 